## Importing Libraries

In [1]:
import pandas as pd

### Importing DataFrames

In [2]:
df_main = pd.read_csv('./cleaned_data/incidents2.csv')
df_names = pd.read_csv('./cleaned_data/names2.csv')
df_census = pd.read_csv('./cleaned_data/counties2.csv')
df_states = pd.read_csv('./data/state_abbrv.csv')

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

### Examining our main dataframe

In [3]:
df_main.head()

Unnamed: 0  Unique ID                   Name Age  Gender  \
0         0.0    25747.0         Mark A. Horton  21    Male   
1         1.0    25748.0  Phillip A. Blurbridge  19    Male   
2         2.0    25746.0        Samuel H. Knapp  17    Male   
3         3.0    25749.0             Mark Ortiz  23    Male   
4         4.0        1.0  LaTanya Janelle McCoy  24  Female   

                      Race    Race with imputations Imputation probability  \
0   African-American/Black   African-American/Black            Not imputed   
1   African-American/Black   African-American/Black            Not imputed   
2  European-American/White  European-American/White            Not imputed   
3          Hispanic/Latino          Hispanic/Latino            Not imputed   
4   African-American/Black   African-American/Black            Not imputed   

   Date of injury resulting in death (month/day/year) State  \
0                                         01/01/2000     MI   
1                                         01/01/2000     MI   
2                                         01/01/2000     CA   
3                                         01/01/2000     NM   
4                                         01/02/2000     CA   

  Location of death (county)        Agency or agencies involved  \
0                      Wayne                               None   
1                      Wayne                               None   
2                  Mendocino  mendocino county sheriff's office   
3                       Eddy       eddy county sheriff's office   
4                 Sacramento       sacramento police department   

  Highest level of force                                  Brief description  \
0                Vehicle  Two Detroit men killed when their car crashed ...   
1                Vehicle  Two Detroit men killed when their car crashed ...   
2                Vehicle  Samuel Knapp was allegedly driving a stolen ve...   
3                Vehicle  A motorcycle was allegedly being driven errati...   
4                Vehicle  LaTanya Janelle McCoy's car was struck from be...   

  Dispositions/Exclusions INTERNAL USE, NOT FOR ANALYSIS  \
0                                         Unreported       
1                                         Unreported       
2                                         Unreported       
3                                         Unreported       
4                                            Unknown       

  Intended use of force (Developing)  \
0                            Pursuit   
1                            Pursuit   
2                            Pursuit   
3                            Pursuit   
4                            Pursuit   

  Foreknowledge of mental illness? INTERNAL USE, NOT FOR ANALYSIS        Date  \
0                                                 No               2000-01-01   
1                                                 No               2000-01-01   
2                                                 No               2000-01-01   
3                                                 No               2000-01-01   
4                                                 No               2000-01-02   

     year  month  week_of_year  day_of_month day_of_week  day_of_year   Agency  
0  2000.0    1.0           0.0           1.0           5          1.0     None  
1  2000.0    1.0           0.0           1.0           5          1.0     None  
2  2000.0    1.0           0.0           1.0           5          1.0  sheriff  
3  2000.0    1.0           0.0           1.0           5          1.0  sheriff  
4  2000.0    1.0           1.0           2.0           6          2.0   police

Due to the uninspired nature of local governments accross the nation, the names of counties may be repeated from state to state. In order to avoid this confusion when trying to aggregating on a county basis down the line, we must create a new location feature which includes the county name, as well the state it exists within. This will also act as our foreign key for our census table, which is aggregated on a county basis.

In [4]:
df_main['county_state'] = df_main['Location of death (county)'] + ', ' + df_main['State']

To create the same format for the primary key within the census dataframe, we must start by removing the word "county" that exists after every county. Accordingly, we have a function that removes that last word from list of words in a phrase. 

In [5]:
def decountyifier(x):
    x1 = x.split()
    x2 = x1[:-1:]
    x3 = " ".join(x2)
    return x3

### Mapping function to every row in the 'name' column of the census dataframe.

In [6]:
df_census['name'] = df_census['name'].map(lambda x: decountyifier(x))

Another problem presents itself as the state identifying data in the incidents dataframe is formatted as a two letter abbreviation where it is the full name of the state in the census dataframe. In order to transform the full name into the abbreviated format, we use a fourth table that only contains the abbreviated state names and full state names, and join it with the census data on the state column. To simplfy the merging process, we reformat the name of the column so they are the same on both dataframes. Finally, we concatenate the strings of the county name and state name (seperated by a space and comma).

### Renaming state column

In [7]:
df_states.rename(columns={'State':'state'}, inplace=True)

### Merging state abbreviations to census dataframes

In [8]:
df_census = pd.merge(df_census, df_states, how="inner", on=['state'])

### Creating county, state column

In [9]:
df_census['county_state'] = df_census['name'] + ', ' + df_census['Code']

Similarly, in order to merge on the incidents dataframe with the last name dataframe we will have to create the same format for the data in both dataframes. This will include breaking out the last names from the name column in the in incidents dataframe, lowering the string case and removing suffixes. Therefore, a function was created that splits a name by term (including suffixes), and returns the last term if it is not a suffix, and the second to last term if the last term is a suffix.

In [10]:
def last_name(x):
    new = []
    x1 = x.split()
    x2 = x1[-1]
    if x2 == 'Jr.':
        return x1[-2]
    elif x2 == 'Sr.':
        return x1[-2]
    elif x2 == 'II':
        return x1[-2]
    elif x2 == 'III':
        return x1[-2]
    elif x2 == 'IV':
        return x1[-2]
    elif x2 == 'V':
        return x1[-2]
    elif x2 == 'VI':
        return x1[-2]
    elif x2 == 'VII':
        return x1[-2]
    else: 
        return x2

We map this function to the data in the name column, and overwrite the previously data.

In [11]:
df_main['last_name'] = df_main['Name'].map(lambda x: last_name(x))

After we apply a blnaket string case lowering method to the entire column.

In [12]:
df_main['last_name'] = df_main['last_name'].str.lower()

Finally, we rename the column last name to make match the column name in names dataframe.

In [13]:
df_names.rename(columns = {'name':'last_name'}, inplace = True)

Again, we apply a string case-lowering method, this time to the last name column from the names dataframe.

In [14]:
df_names['last_name'] = df_names['last_name'].str.lower()

### Merging dataframes

Merging incidents dataframe to census dataframe

In [15]:
df_main = pd.merge(df_main, df_census, how='inner', on = ['county_state'])

Merging incidents dataframe to last names dataframe

In [16]:
df_main = pd.merge(df_main, df_names, how='inner', on = ['last_name'])

### Displaying the full, newly created dataframe

In [17]:
df_main

Unnamed: 0_x  Unique ID                                          Name  \
0               0.0    25747.0                                Mark A. Horton   
1           25492.0    25227.0                                  Andre Horton   
2           16483.0    13379.0                                 Marlon Horton   
3            3948.0     3153.0  Donnie Clay Coleman Jr. aka Leon Eric Horton   
4           12134.0     9594.0                        Kenneth Michael Horton   
...             ...        ...                                           ...   
26002       30491.0    30480.0                              Gary Lee Deering   
26003       30502.0    30491.0                        David Ronald Bridgette   
26004       30507.0    30496.0                           William Dean Hewett   
26005       30615.0    30605.0                          Kristopher M. Burden   
26006       30747.0    30746.0                  Marvin Richard Hitchcock Jr.   

        Age Gender                     Race    Race with imputations  \
0        21   Male   African-American/Black   African-American/Black   
1      42.0   Male   African-American/Black   African-American/Black   
2      28.0   Male   African-American/Black   African-American/Black   
3        27   Male   African-American/Black   African-American/Black   
4        31   Male   African-American/Black   African-American/Black   
...     ...    ...                      ...                      ...   
26002  74.0   Male  European-American/White  European-American/White   
26003  45.0   Male  European-American/White  European-American/White   
26004  54.0   Male  European-American/White  European-American/White   
26005  34.0   Male  European-American/White  European-American/White   
26006  71.0   Male  European-American/White  European-American/White   

      Imputation probability  \
0                Not imputed   
1                Not imputed   
2                Not imputed   
3                Not imputed   
4                Not imputed   
...                      ...   
26002            Not imputed   
26003            Not imputed   
26004            Not imputed   
26005            Not imputed   
26006            Not imputed   

       Date of injury resulting in death (month/day/year) State  \
0                                             01/01/2000     MI   
1                                             12/13/2018     TN   
2                                             09/07/2013     IL   
3                                             02/09/2004     CA   
4                                             11/04/2010     TX   
...                                                  ...    ...   
26002                                         06/23/2021     MO   
26003                                         06/24/2021     MI   
26004                                         06/24/2021     NC   
26005                                         07/15/2021     KY   
26006                                         08/09/2021     NC   

      Location of death (county)  \
0                          Wayne   
1                         Shelby   
2                           Cook   
3                           Kern   
4                         Dallas   
...                          ...   
26002                       Dade   
26003                       Iron   
26004                   Columbus   
26005                   Edmonson   
26006                    Pamlico   

                             Agency or agencies involved  \
0                                                   None   
1                              memphis police department   
2                              chicago police department   
3                           kern county sheriff's office   
4                               dallas police department   
...                                                  ...   
26002                       dade county sheriff's office   
26003  iron river police department, iron county sher...   
26004             

### Checking the full dataframe for null values

In [18]:
def null_sort(df):
     return df.apply(pd.isnull).sum().sort_values(ascending=False).loc[lambda series: series>0]

In [19]:
new = null_sort(df_main)

In [20]:
new = pd.DataFrame(new)

In [21]:
new

Empty DataFrame
Columns: [0]
Index: []

### Renaming dataframe

In [22]:
df = df_main

### Casting day of week and age columns as float types

In [23]:
df['day_of_week'] = df['day_of_week'].astype('float')

In [24]:
df['Age'] = df['Age'].astype('float')

### Splitting our categorical and numerical variables

In [25]:
df_cat = df.select_dtypes(include='object')
df_num = df.select_dtypes(exclude='object')

### Defining a function which returns the number of unique values in each column of a dataframe

In [26]:
def unq_cnt(df):
    new_list = []
    for i in df.columns:
        new_list.append(i)
        def un_cnt(col):
            cnt = (len(df[col].unique()))
            return cnt
    col_counts = [un_cnt(i) for i in df.columns] 
    return list(zip(new_list,col_counts))

### Creating a dataframe based on the column unique value count of cat. variables

In [27]:
df_unq_cat = pd.DataFrame(unq_cnt(df_cat))

In [28]:
df_unq_cat.sort_values(by = 1, ascending = False)

0      1
0                                                Name  25556
10                                  Brief description  24603
17                                          last_name   9516
5    Date of injury resulting in death (month/day/...   7427
14                                               Date   7427
4                              Imputation probability   6216
8                         Agency or agencies involved   6040
16                                       county_state   2277
18                                               name   1420
7                          Location of death (county)   1420
11  Dispositions/Exclusions INTERNAL USE, NOT FOR ...    135
6                                               State     50
19                                              state     50
20                                             Abbrev     50
21                                               Code     50
9                              Highest level of force     17
12                 Intended use of force (Developing)      8
3                               Race with imputations      8
2                                                Race      7
15                                             Agency      5
13  Foreknowledge of mental illness? INTERNAL USE,...      4
1                                              Gender      3

### Dropping unnecessary categorical columns

In [29]:
df_cat = df_cat.drop(columns = ['Name', 
                       'last_name', 
                       ' Date of injury resulting in death (month/day/year)',
                       'Date',
                       'Imputation probability', 
                       'Agency or agencies involved',
                       'county_state',
                       'name',
                       'Location of death (county)',
                       'Race with imputations', 
                       'Dispositions/Exclusions INTERNAL USE, NOT FOR ANALYSIS',
                       'state',
                       'Race',
                       'Abbrev',
                       'Code', 'Brief description'])

### Checking unique values after dropping columns

In [30]:
df_unq_cat = pd.DataFrame(unq_cnt(df_cat))

In [31]:
df_unq_cat.sort_values(by = 1, ascending = False)

0   1
1                                              State  50
2                             Highest level of force  17
3                 Intended use of force (Developing)   8
5                                             Agency   5
4  Foreknowledge of mental illness? INTERNAL USE,...   4
0                                             Gender   3

### Dummifying remaining categorical variables

In [32]:
df_cat = pd.get_dummies(df_cat, drop_first=True)

### Creating a dataframe based on the column unique value count of num. variables

In [33]:
df_unq_num = pd.DataFrame(unq_cnt(df_num))

In [34]:
df_unq_num.sort_values(by = 1, ascending = False)

0      1
0                                      Unnamed: 0_x  25750
1                                         Unique ID  25750
186                                    cum_prop100k   9516
182                                      Unnamed: 0   9516
184                                           count   6548
183                                            rank   6548
187                                        pctwhite   4472
188                                        pctblack   3148
9                                      Unnamed: 0_y   2283
69                           per_capita_income_2017   2282
80                                fed_spending_2009   2277
77                                       sales_2007   2276
11                                          pop2010   2270
15                                          pop2014   2266
17                                          pop2016   2263
167                                        pop_2019   2262
12                                          pop2011   2261
14                                          pop2013   2261
13                                          pop2012   2261
18                                          pop2017   2260
81                                        area_2010   2260
10                                          pop2000   2258
16                                          pop2015   2257
108                       civilian_labor_force_2012   2249
109                                   employed_2012   2248
132                                   employed_2017   2246
96                        civilian_labor_force_2009   2245
92                        civilian_labor_force_2008   2244
121                                   employed_2015   2242
113                                   employed_2013   2242
60                               housing_units_2010   2242
131                       civilian_labor_force_2017   2242
100                       civilian_labor_force_2010   2241
112                       civilian_labor_force_2013   2240
120                       civilian_labor_force_2015   2240
125                                   employed_2016   2239
97                                    employed_2009   2238
116                       civilian_labor_force_2014   2237
157                      mean_household_income_2019   2237
88                        civilian_labor_force_2007   2235
124                       civilian_labor_force_2016   2235
147                                 households_2019   2233
117                                   employed_2014   2231
105                                   employed_2011   2231
65                                  households_2017   2230
104                       civilian_labor_force_2011   2228
101                                   employed_2010   2227
93                                    employed_2008   2226
159                    median_household_income_2019   2225
64                                  households_2010   2225
89                                    employed_2007   2220
74                  private_nonfarm_employment_2009   2214
72                     median_household_income_2017   2211
71                     median_household_income_2016   2211
70                     median_household_income_2010   2181
165                          per_capita_income_2019   2156
161        median_individual_income_age_25plus_2019   2144
68                           per_capita_income_2010   2138
160                   median_individual_income_2019   2133
78                            sales_per_capita_2007   2119
54                                    veterans_2010   2065
76                nonemployment_establishments_2009   1989
98                                  unemployed_2009   1906
102                                 unemployed_2010   1895
106                                 unemployed_2011   1885
185                                        prop100k   1883
110                                 unemployed_2012   1850
40                          white_not_

### Dropping unnecessary numeric columns 

In [35]:
df_num = df_num.drop(columns = ['Unnamed: 0_x', 'Unnamed: 0', 'Unnamed: 0_y'])

### Examining numeric variable dataframe 

In [36]:
df_num.head()

Unique ID   Age    year  month  week_of_year  day_of_month  day_of_week  \
0    25747.0  21.0  2000.0    1.0           0.0           1.0          5.0   
1    25227.0  42.0  2018.0   12.0          49.0          13.0          3.0   
2    13379.0  28.0  2013.0    9.0          35.0           7.0          5.0   
3     3153.0  27.0  2004.0    2.0           6.0           9.0          0.0   
4     9594.0  31.0  2010.0   11.0          44.0           4.0          3.0   

   day_of_year    pop2000  pop2010    pop2011    pop2012    pop2013  \
0          1.0  2061162.0  1820584  1802164.0  1794073.0  1777790.0   
1        347.0   897472.0   927644   933268.0   939425.0   938825.0   
2        250.0  5376741.0  5194675  5217049.0  5237174.0  5250498.0   
3         40.0   661645.0   839631   848767.0   855237.0   864014.0   
4        308.0  2218899.0  2368139  2408133.0  2454781.0  2483807.0   

     pop2014    pop2015    pop2016    pop2017  age_under_5_2010  \
0  1769007.0  1762098.0  1756598.0  1753616.0               6.5   
1   938434.0   937885.0   937130.0   936961.0               7.2   
2  5253756.0  5245831.0  5231356.0  5211263.0               6.6   
3   871895.0   879607.0   885086.0   893119.0               8.7   
4  2517417.0  2554233.0  2587462.0  2618148.0               8.1   

   age_under_5_2017  age_under_18_2010  age_over_65_2010  age_over_65_2017  \
0               6.5               25.4              12.7              14.4   
1               7.2               26.4              10.3              12.2   
2               6.3               23.7              11.9              13.5   
3               8.2               30.3               9.0              10.2   
4               7.6               27.6               8.8              10.0   

   median_age_2017  female_2010  white_2010  black_2010  black_2017  \
0             37.9         52.0        52.3        40.5       19.55   
1             35.3         52.3        40.6        52.1       26.68   
2             36.4         51.6        55.4        24.8       11.86   
3             31.3         48.4        59.5         5.8        2.74   
4             33.3         50.6        53.5        22.3       11.22   

   native_2010  native_2017  asian_2010  asian_2017  pac_isl_2017  \
0          0.4         0.16         2.5        1.56          0.01   
1          0.2         0.08         2.3        1.26          0.02   
2          0.4         0.13         6.2        3.49          0.02   
3          1.5         0.55         4.2        2.35          0.09   
4          0.7         0.16         5.0        3.00          0.03   

   other_single_race_2017  two_plus_races_2010  two_plus_races_2017  \
0                    0.97                  2.4                 1.17   
1                    1.38                  1.4                 0.87   
2                    4.94                  2.5                 1.25   
3                    5.03                  4.5                 1.68   
4                    3.58                  2.8                 1.35   

   hispanic_2010  hispanic_2017  white_not_hispanic_2010  \
0            5.2           5.72                     49.6   
1            5.6           6.10                     38.7   
2           24.0          25.05                     43.9   
3           49.2          52.22                     38.6   
4           38.3          39.63                     33.1   

   white_not_hispanic_2017  speak_english_only_2017  \
0                    49.68                     86.0   
1                    36.45                     90.7   
2                    42.68                     64.9   
3                    35.35                     55.9   
4                    30.22                     57.4   

   no_move_in_one_plus_year_2010  foreign_born_2010  \
0                           85.4                7.6   
1                           81.6                6.0   
2                           85.8               21.0   
3                           79.4               20.5   

### Concatenating numeric variable dataframe and numeric dataframe

In [37]:
df_new = pd.concat([df_num, df_cat], axis = 1)

### Examining newly created full dataframe, with both numeric and categorical variables

In [38]:
df_new.head()

Unique ID   Age    year  month  week_of_year  day_of_month  day_of_week  \
0    25747.0  21.0  2000.0    1.0           0.0           1.0          5.0   
1    25227.0  42.0  2018.0   12.0          49.0          13.0          3.0   
2    13379.0  28.0  2013.0    9.0          35.0           7.0          5.0   
3     3153.0  27.0  2004.0    2.0           6.0           9.0          0.0   
4     9594.0  31.0  2010.0   11.0          44.0           4.0          3.0   

   day_of_year    pop2000  pop2010    pop2011    pop2012    pop2013  \
0          1.0  2061162.0  1820584  1802164.0  1794073.0  1777790.0   
1        347.0   897472.0   927644   933268.0   939425.0   938825.0   
2        250.0  5376741.0  5194675  5217049.0  5237174.0  5250498.0   
3         40.0   661645.0   839631   848767.0   855237.0   864014.0   
4        308.0  2218899.0  2368139  2408133.0  2454781.0  2483807.0   

     pop2014    pop2015    pop2016    pop2017  age_under_5_2010  \
0  1769007.0  1762098.0  1756598.0  1753616.0               6.5   
1   938434.0   937885.0   937130.0   936961.0               7.2   
2  5253756.0  5245831.0  5231356.0  5211263.0               6.6   
3   871895.0   879607.0   885086.0   893119.0               8.7   
4  2517417.0  2554233.0  2587462.0  2618148.0               8.1   

   age_under_5_2017  age_under_18_2010  age_over_65_2010  age_over_65_2017  \
0               6.5               25.4              12.7              14.4   
1               7.2               26.4              10.3              12.2   
2               6.3               23.7              11.9              13.5   
3               8.2               30.3               9.0              10.2   
4               7.6               27.6               8.8              10.0   

   median_age_2017  female_2010  white_2010  black_2010  black_2017  \
0             37.9         52.0        52.3        40.5       19.55   
1             35.3         52.3        40.6        52.1       26.68   
2             36.4         51.6        55.4        24.8       11.86   
3             31.3         48.4        59.5         5.8        2.74   
4             33.3         50.6        53.5        22.3       11.22   

   native_2010  native_2017  asian_2010  asian_2017  pac_isl_2017  \
0          0.4         0.16         2.5        1.56          0.01   
1          0.2         0.08         2.3        1.26          0.02   
2          0.4         0.13         6.2        3.49          0.02   
3          1.5         0.55         4.2        2.35          0.09   
4          0.7         0.16         5.0        3.00          0.03   

   other_single_race_2017  two_plus_races_2010  two_plus_races_2017  \
0                    0.97                  2.4                 1.17   
1                    1.38                  1.4                 0.87   
2                    4.94                  2.5                 1.25   
3                    5.03                  4.5                 1.68   
4                    3.58                  2.8                 1.35   

   hispanic_2010  hispanic_2017  white_not_hispanic_2010  \
0            5.2           5.72                     49.6   
1            5.6           6.10                     38.7   
2           24.0          25.05                     43.9   
3           49.2          52.22                     38.6   
4           38.3          39.63                     33.1   

   white_not_hispanic_2017  speak_english_only_2017  \
0                    49.68                     86.0   
1                    36.45                     90.7   
2                    42.68                     64.9   
3                    35.35                     55.9   
4                    30.22                     57.4   

   no_move_in_one_plus_year_2010  foreign_born_2010  \
0                           85.4                7.6   
1                           81.6                6.0   
2                           85.8               21.0   
3                           79.4               20.5   

### Reattaching columns description and race columns

In [39]:
df_new['description'] = df['Brief description']
df_new['Race'] = df['Race']

In [40]:
df_new.head()

Unique ID   Age    year  month  week_of_year  day_of_month  day_of_week  \
0    25747.0  21.0  2000.0    1.0           0.0           1.0          5.0   
1    25227.0  42.0  2018.0   12.0          49.0          13.0          3.0   
2    13379.0  28.0  2013.0    9.0          35.0           7.0          5.0   
3     3153.0  27.0  2004.0    2.0           6.0           9.0          0.0   
4     9594.0  31.0  2010.0   11.0          44.0           4.0          3.0   

   day_of_year    pop2000  pop2010    pop2011    pop2012    pop2013  \
0          1.0  2061162.0  1820584  1802164.0  1794073.0  1777790.0   
1        347.0   897472.0   927644   933268.0   939425.0   938825.0   
2        250.0  5376741.0  5194675  5217049.0  5237174.0  5250498.0   
3         40.0   661645.0   839631   848767.0   855237.0   864014.0   
4        308.0  2218899.0  2368139  2408133.0  2454781.0  2483807.0   

     pop2014    pop2015    pop2016    pop2017  age_under_5_2010  \
0  1769007.0  1762098.0  1756598.0  1753616.0               6.5   
1   938434.0   937885.0   937130.0   936961.0               7.2   
2  5253756.0  5245831.0  5231356.0  5211263.0               6.6   
3   871895.0   879607.0   885086.0   893119.0               8.7   
4  2517417.0  2554233.0  2587462.0  2618148.0               8.1   

   age_under_5_2017  age_under_18_2010  age_over_65_2010  age_over_65_2017  \
0               6.5               25.4              12.7              14.4   
1               7.2               26.4              10.3              12.2   
2               6.3               23.7              11.9              13.5   
3               8.2               30.3               9.0              10.2   
4               7.6               27.6               8.8              10.0   

   median_age_2017  female_2010  white_2010  black_2010  black_2017  \
0             37.9         52.0        52.3        40.5       19.55   
1             35.3         52.3        40.6        52.1       26.68   
2             36.4         51.6        55.4        24.8       11.86   
3             31.3         48.4        59.5         5.8        2.74   
4             33.3         50.6        53.5        22.3       11.22   

   native_2010  native_2017  asian_2010  asian_2017  pac_isl_2017  \
0          0.4         0.16         2.5        1.56          0.01   
1          0.2         0.08         2.3        1.26          0.02   
2          0.4         0.13         6.2        3.49          0.02   
3          1.5         0.55         4.2        2.35          0.09   
4          0.7         0.16         5.0        3.00          0.03   

   other_single_race_2017  two_plus_races_2010  two_plus_races_2017  \
0                    0.97                  2.4                 1.17   
1                    1.38                  1.4                 0.87   
2                    4.94                  2.5                 1.25   
3                    5.03                  4.5                 1.68   
4                    3.58                  2.8                 1.35   

   hispanic_2010  hispanic_2017  white_not_hispanic_2010  \
0            5.2           5.72                     49.6   
1            5.6           6.10                     38.7   
2           24.0          25.05                     43.9   
3           49.2          52.22                     38.6   
4           38.3          39.63                     33.1   

   white_not_hispanic_2017  speak_english_only_2017  \
0                    49.68                     86.0   
1                    36.45                     90.7   
2                    42.68                     64.9   
3                    35.35                     55.9   
4                    30.22                     57.4   

   no_move_in_one_plus_year_2010  foreign_born_2010  \
0                           85.4                7.6   
1                           81.6                6.0   
2                           85.8               21.0   
3                           79.4               20.5   

### Setting the index equal to the Unique ID number

In [41]:
df_new.set_index(['Unique ID'], inplace = True)

### Sorting index

In [42]:
df_new.sort_index(inplace = True)

### Exporting full dataset to CSV file

In [43]:
df_new.to_csv('./cleaned_data/full.csv')

### Creating training (labeled) data by removing rows where race was unspecified

In [44]:
train_set = df_new[df_new['Race'] != 'Race unspecified']

### Examining training data

In [45]:
train_set.head()

Age    year  month  week_of_year  day_of_month  day_of_week  \
Unique ID                                                                 
1.0        24.0  2000.0    1.0           1.0           2.0          6.0   
4.0        45.0  2000.0    1.0           1.0           5.0          2.0   
5.0        20.0  2000.0    1.0           1.0           5.0          2.0   
6.0        19.0  2000.0    1.0           1.0           6.0          3.0   
14.0       26.0  2000.0    1.0           2.0          10.0          0.0   

           day_of_year    pop2000  pop2010    pop2011    pop2012    pop2013  \
Unique ID                                                                     
1.0                2.0  1223499.0  1418788  1434506.0  1446585.0  1459474.0   
4.0                5.0    88787.0   101547   102468.0   103362.0   103625.0   
5.0                5.0   816006.0   920581   948809.0   974918.0   982930.0   
6.0                6.0    14598.0    13665    13787.0    13841.0    13858.0   
14.0              10.0  1517550.0  1526006  1539649.0  1551944.0  1558109.0   

             pop2014    pop2015    pop2016    pop2017  age_under_5_2010  \
Unique ID                                                                 
1.0        1477522.0  1496130.0  1513260.0  1530615.0               7.1   
4.0         104148.0   104249.0   104173.0   104346.0               6.6   
5.0         994658.0  1008841.0  1024248.0  1041423.0               6.8   
6.0          13924.0    13810.0    13761.0    13806.0               6.4   
14.0       1564042.0  1570507.0  1574765.0  1580863.0               6.6   

           age_under_5_2017  age_under_18_2010  age_over_65_2010  \
Unique ID                                                          
1.0                     6.6               25.6              11.2   
4.0                     6.1               24.5              14.5   
5.0                     6.2               23.9               9.1   
6.0                     5.9               22.4              18.2   
14.0                    6.9               22.5              12.1   

           age_over_65_2017  median_age_2017  female_2010  white_2010  \
Unique ID                                                               
1.0                    13.0             35.9         51.0        57.5   
4.0                    16.5             39.6         52.0        70.0   
5.0                    10.8             35.2         51.3        44.5   
6.0                    19.2             40.9         51.2        95.0   
14.0                   12.9             34.1         52.8        41.0   

           black_2010  black_2017  native_2010  native_2017  asian_2010  \
Unique ID                                                                 
1.0              10.4        4.93          1.0         0.35        14.3   
4.0              25.8       13.37          0.4         0.19         0.8   
5.0              44.1       22.07          0.2         0.19         5.6   
6.0               1.2        0.60          0.4         0.17         0.4   
14.0             43.4       21.28          0.5         0.18         6.3   

           asian_2017  pac_isl_2017  other_single_race_2017  \
Unique ID                                                     
1.0              7.67          0.54                    3.65   
4.0              0.43          0.01                    0.28   
5.0              3.34          0.02                    0.70   
6.0              0.13          0.00                    0.61   
14.0             3.55          0.03                    2.78   

           two_plus_races_2010  two_plus_races_2017  hispanic_2010  \
Unique ID                                                            
1.0                        6.6                 3.52           21.6   
4.0                        1.7                 1.00            2.9   
5.0                        2.2                 1.17            7.9   
6.0                        1.0                 1.07            4.1   
14.0                       2.

### Exporting training data to CSV file

In [46]:
train_set.to_csv('./data/train.csv')

### Creating training (unlabeled) data by only keeping rows where race was unspecified

In [47]:
test_set = df_new[df_new['Race'] == 'Race unspecified']

### Examining testing data

In [48]:
test_set.head()

Age    year  month  week_of_year  day_of_month  day_of_week  \
Unique ID                                                                 
2.0        53.0  2000.0    1.0           1.0           2.0          6.0   
3.0        23.0  2000.0    1.0           1.0           5.0          2.0   
7.0        23.0  2000.0    1.0           1.0           6.0          3.0   
8.0        31.0  2000.0    1.0           1.0           6.0          3.0   
10.0       28.0  2000.0    1.0           1.0           7.0          4.0   

           day_of_year    pop2000  pop2010    pop2011    pop2012     pop2013  \
Unique ID                                                                      
2.0                2.0   665865.0   691893   699684.0   710604.0    717321.0   
3.0                5.0    39678.0    38950    38799.0    38508.0     37389.0   
7.0                6.0   695454.0   919628   944943.0   968204.0    991619.0   
8.0                6.0  1709434.0  2035210  2062381.0  2076601.0   2086732.0   
10.0               7.0  9519338.0  9818605  9885998.0  9956152.0  10018604.0   

              pop2014     pop2015     pop2016     pop2017  age_under_5_2010  \
Unique ID                                                                     
2.0          725647.0    737418.0    746690.0    753253.0               7.3   
3.0           37012.0     36668.0     36475.0     36518.0               5.7   
7.0         1011315.0   1034442.0   1057237.0   1076837.0               7.4   
8.0         2104038.0   2120794.0   2137131.0   2157404.0               7.8   
10.0       10072695.0  10123248.0  10150558.0  10163507.0               6.6   

           age_under_5_2017  age_under_18_2010  age_over_65_2010  \
Unique ID                                                          
2.0                     7.2               23.9               9.0   
3.0                     5.7               22.5              15.8   
7.0                     6.9               25.4               8.8   
8.0                     7.3               29.2               8.9   
10.0                    6.3               24.5              10.9   

           age_over_65_2017  median_age_2017  female_2010  white_2010  \
Unique ID                                                               
2.0                    11.1             35.5         52.1        33.3   
3.0                    17.6             41.6         49.9        91.8   
7.0                    10.4             34.9         51.6        55.3   
8.0                    10.6             32.9         50.3        56.7   
10.0                   12.5             36.0         50.7        50.3   

           black_2010  black_2017  native_2010  native_2017  asian_2010  \
Unique ID                                                                 
2.0              54.3       26.99          0.4         0.33         5.1   
3.0               4.9        2.04          0.2         0.01         0.5   
7.0              30.8       15.59          0.5         0.15         4.6   
8.0               8.9        4.21          1.1         0.40         6.3   
10.0              8.7        4.10          0.7         0.34        13.7   

           asian_2017  pac_isl_2017  other_single_race_2017  \
Unique ID                                                     
2.0              3.03          0.02                    0.98   
3.0              0.33          0.00                    0.31   
7.0              2.75          0.03                    2.49   
8.0              3.46          0.16                    8.49   
10.0             7.23          0.14                   10.40   

           two_plus_races_2010  two_plus_races_2017  hispanic_2010  \
Unique ID                                                            
2.0                        2.4                 1.33            9.8   
3.0                        1.3                 0.90            3.9   
7.0                        2.6                 1.44           12.2   
8.0                        5.0                 2.34           49

### Exporting training data to CSV file

In [49]:
test_set.to_csv('./data/test.csv')